In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
def fisher_score(X, y):
    labels= np.unique(y)
    mean_overall= np.mean(X, axis=0)
    fisher_scores= []
    for i in range(X.shape[1]):
        numerator= 0
        denominator= 0
        for label in labels:
            X_label= X[y== label]
            mean_label= np.mean(X_label[:, i])
            numerator += len(X_label) * (mean_label - mean_overall[i])**2
            denominator += np.sum((X_label[:, i] - mean_label)**2)
        fisher_scores.append(numerator / (denominator + 1e-10))
    return np.array(fisher_scores)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [7]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [8]:
fisher_scores= fisher_score(X_sm_tl, y_sm_tl)
k= 10
top_k_indices= np.argsort(fisher_scores)[-k:]

X_sm_tl_selected= X_sm_tl[:, top_k_indices]
X_sm_tl_test_selected= X_test[:, top_k_indices]

In [9]:
feature_names = X.columns
fisher_df= pd.DataFrame({'Feature': feature_names,
                          'Fisher Score': fisher_scores})

fisher_df= fisher_df.sort_values(by= 'Fisher Score', ascending= False)
fisher_df

,Feature,Fisher Score
20,HBP,0.000943
2,Cholesterol,0.000728
19,Sleep Hours Per Day,0.000719
18,Physical Activity Days Per Week,0.000645
7,Obesity,0.000541
14,Sedentary Hours Per Day,0.000504
17,Triglycerides,0.000465
10,Diet,0.000462
8,Alcohol Consumption,0.000410
5,Family History,0.000320


--- LogisticRegression ---

In [10]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1125
           1       0.35      0.48      0.40       628

    accuracy                           0.49      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.34597701149425286
Accuracy: 0.48887621220764405
AUC: 0.4820396319886765


--- DecisionTreeClassifier ---

In [11]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.65      0.63      0.64      1125
           1       0.37      0.39      0.38       628

    accuracy                           0.54      1753
   macro avg       0.51      0.51      0.51      1753
weighted avg       0.55      0.54      0.54      1753

Precision: 0.36786786786786785
Accuracy: 0.5413576725613235
AUC: 0.5079525831564048


--- Tuned - DecisionTreeClassifier ---

In [12]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 20}
Best Precision Score: 0.6385424295298758


In [13]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.65      0.95      0.77      1125
           1       0.43      0.07      0.12       628

    accuracy                           0.63      1753
   macro avg       0.54      0.51      0.44      1753
weighted avg       0.57      0.63      0.54      1753

Precision: 0.43434343434343436
Accuracy: 0.6343411294922989
AUC: 0.5107430997876858


--- KNeighborsClassifier ---

In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.63      0.53      0.58      1125
           1       0.35      0.45      0.39       628

    accuracy                           0.50      1753
   macro avg       0.49      0.49      0.49      1753
weighted avg       0.53      0.50      0.51      1753

Precision: 0.3488943488943489
Accuracy: 0.5014261266400456
AUC: 0.4870785562632697


--- Tuned - KNeighborsClassifier ---

In [15]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.6895178964255761


In [16]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      1125
           1       0.35      0.19      0.24       628

    accuracy                           0.58      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.58      0.54      1753

Precision: 0.34615384615384615
Accuracy: 0.5824301197946378
AUC: 0.4976702052370842


--- GaussianNB ---

In [17]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.63      0.51      0.56      1125
           1       0.34      0.46      0.39       628

    accuracy                           0.49      1753
   macro avg       0.48      0.48      0.48      1753
weighted avg       0.52      0.49      0.50      1753

Precision: 0.3420738974970203
Accuracy: 0.4905875641756988
AUC: 0.4768619957537155


--- SVM ---

In [18]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.63      0.72      0.67      1125
           1       0.31      0.22      0.26       628

    accuracy                           0.55      1753
   macro avg       0.47      0.47      0.47      1753
weighted avg       0.51      0.55      0.52      1753

Precision: 0.31263858093126384
Accuracy: 0.5453508271534512
AUC: 0.4888188251946214


--- Random Forest ---

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [20]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.64      0.86      0.73      1125
           1       0.36      0.14      0.20       628

    accuracy                           0.60      1753
   macro avg       0.50      0.50      0.47      1753
weighted avg       0.54      0.60      0.54      1753

Precision: 0.356
Accuracy: 0.6006845407872219
AUC: 0.506557678697806


In [21]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
Best Precision Score: 0.7506317076127752


In [22]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.64      0.87      0.74      1125
           1       0.34      0.12      0.17       628

    accuracy                           0.60      1753
   macro avg       0.49      0.49      0.46      1753
weighted avg       0.53      0.60      0.54      1753

Precision: 0.33636363636363636
Accuracy: 0.6006845407872219
AUC: 0.49900070771408345


--- AdaBoost ---

In [23]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [24]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.63      0.71      0.67      1125
           1       0.34      0.27      0.30       628

    accuracy                           0.55      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.55      0.54      1753

Precision: 0.338
Accuracy: 0.549343981745579
AUC: 0.486617126680821


In [25]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.7485840283438058


In [26]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.64      0.94      0.76      1125
           1       0.31      0.05      0.09       628

    accuracy                           0.62      1753
   macro avg       0.47      0.49      0.42      1753
weighted avg       0.52      0.62      0.52      1753

Precision: 0.31
Accuracy: 0.6200798630918426
AUC: 0.4893842887473461


--- GradientBoosting ---

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.39      0.03      0.05       628

    accuracy                           0.64      1753
   macro avg       0.52      0.50      0.41      1753
weighted avg       0.55      0.64      0.51      1753

Precision: 0.3902439024390244
Accuracy: 0.6366229321163719
AUC: 0.48452512384996466


In [28]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.772585566910753


In [29]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.64      0.98      0.78      1125
           1       0.39      0.03      0.05       628

    accuracy                           0.64      1753
   macro avg       0.52      0.50      0.41      1753
weighted avg       0.55      0.64      0.51      1753

Precision: 0.3902439024390244
Accuracy: 0.6366229321163719
AUC: 0.48452512384996466


--- XGB ---

In [30]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.64      0.82      0.72      1125
           1       0.37      0.19      0.25       628

    accuracy                           0.59      1753
   macro avg       0.51      0.50      0.48      1753
weighted avg       0.54      0.59      0.55      1753

Precision: 0.3667711598746082
Accuracy: 0.5932686822589845
AUC: 0.4973460721868366


In [31]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7135871093129518
              precision    recall  f1-score   support

           0       0.65      0.91      0.76      1125
           1       0.41      0.11      0.17       628

    accuracy                           0.62      1753
   macro avg       0.53      0.51      0.46      1753
weighted avg       0.56      0.62      0.55      1753

Precision: 0.40588235294117647
Accuracy: 0.623502567027952
AUC: 0.5053262561924982


--- LGBM ---

In [32]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2438
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.64      0.90      0.75      1125
           1       0.34      0.09      0.15       628

    accuracy                           0.61      1753
   macro avg       0.49      0.50      0.45      1753
weighted avg       0.53      0.61      0.53      1753

Precision: 0.3430232558139535
Accuracy: 0.6109526525955505
AUC: 0.4901316348195329


In [33]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000694 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2438
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 50, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.7983442076752676
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_le

--- CatBoost ---

In [34]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.02579
0:	learn: 0.6896744	total: 150ms	remaining: 2m 30s
1:	learn: 0.6858032	total: 155ms	remaining: 1m 17s
2:	learn: 0.6816484	total: 163ms	remaining: 54.3s
3:	learn: 0.6792786	total: 169ms	remaining: 42s
4:	learn: 0.6767961	total: 174ms	remaining: 34.6s
5:	learn: 0.6741275	total: 180ms	remaining: 29.9s
6:	learn: 0.6707069	total: 189ms	remaining: 26.8s
7:	learn: 0.6671767	total: 195ms	remaining: 24.2s
8:	learn: 0.6642935	total: 202ms	remaining: 22.3s
9:	learn: 0.6610170	total: 209ms	remaining: 20.7s
10:	learn: 0.6563439	total: 214ms	remaining: 19.3s
11:	learn: 0.6541491	total: 220ms	remaining: 18.1s
12:	learn: 0.6499277	total: 225ms	remaining: 17.1s
13:	learn: 0.6476165	total: 232ms	remaining: 16.3s
14:	learn: 0.6452189	total: 237ms	remaining: 15.6s
15:	learn: 0.6431613	total: 242ms	remaining: 14.9s
16:	learn: 0.6410733	total: 247ms	remaining: 14.3s
17:	learn: 0.6392458	total: 254ms	remaining: 13.8s
18:	learn: 0.6376803	total: 259ms	remaining: 13.4s
19:	learn: 0

160:	learn: 0.5372777	total: 1.05s	remaining: 5.46s
161:	learn: 0.5370366	total: 1.06s	remaining: 5.47s
162:	learn: 0.5367441	total: 1.07s	remaining: 5.49s
163:	learn: 0.5364736	total: 1.08s	remaining: 5.49s
164:	learn: 0.5358730	total: 1.08s	remaining: 5.47s
165:	learn: 0.5356694	total: 1.09s	remaining: 5.46s
166:	learn: 0.5354743	total: 1.09s	remaining: 5.45s
167:	learn: 0.5352569	total: 1.1s	remaining: 5.43s
168:	learn: 0.5350165	total: 1.1s	remaining: 5.41s
169:	learn: 0.5347586	total: 1.11s	remaining: 5.4s
170:	learn: 0.5346274	total: 1.11s	remaining: 5.39s
171:	learn: 0.5342674	total: 1.12s	remaining: 5.38s
172:	learn: 0.5337062	total: 1.12s	remaining: 5.38s
173:	learn: 0.5333534	total: 1.13s	remaining: 5.36s
174:	learn: 0.5331977	total: 1.13s	remaining: 5.35s
175:	learn: 0.5330398	total: 1.14s	remaining: 5.34s
176:	learn: 0.5328935	total: 1.14s	remaining: 5.32s
177:	learn: 0.5327349	total: 1.15s	remaining: 5.31s
178:	learn: 0.5326120	total: 1.16s	remaining: 5.3s
179:	learn: 0.53

320:	learn: 0.5031681	total: 1.95s	remaining: 4.13s
321:	learn: 0.5030245	total: 1.96s	remaining: 4.12s
322:	learn: 0.5028188	total: 1.96s	remaining: 4.11s
323:	learn: 0.5026961	total: 1.97s	remaining: 4.1s
324:	learn: 0.5025868	total: 1.97s	remaining: 4.09s
325:	learn: 0.5024855	total: 1.98s	remaining: 4.08s
326:	learn: 0.5021938	total: 1.98s	remaining: 4.08s
327:	learn: 0.5017911	total: 1.99s	remaining: 4.07s
328:	learn: 0.5016610	total: 1.99s	remaining: 4.06s
329:	learn: 0.5014604	total: 2s	remaining: 4.05s
330:	learn: 0.5013181	total: 2s	remaining: 4.05s
331:	learn: 0.5009940	total: 2.01s	remaining: 4.04s
332:	learn: 0.5007220	total: 2.01s	remaining: 4.03s
333:	learn: 0.5004617	total: 2.02s	remaining: 4.02s
334:	learn: 0.5002645	total: 2.02s	remaining: 4.02s
335:	learn: 0.5001311	total: 2.03s	remaining: 4.01s
336:	learn: 0.4999299	total: 2.04s	remaining: 4s
337:	learn: 0.4995965	total: 2.04s	remaining: 4s
338:	learn: 0.4993449	total: 2.04s	remaining: 3.99s
339:	learn: 0.4991235	tot

503:	learn: 0.4680413	total: 2.85s	remaining: 2.8s
504:	learn: 0.4679314	total: 2.85s	remaining: 2.79s
505:	learn: 0.4677814	total: 2.85s	remaining: 2.79s
506:	learn: 0.4676222	total: 2.86s	remaining: 2.78s
507:	learn: 0.4674394	total: 2.86s	remaining: 2.77s
508:	learn: 0.4672613	total: 2.87s	remaining: 2.77s
509:	learn: 0.4671409	total: 2.87s	remaining: 2.76s
510:	learn: 0.4669901	total: 2.88s	remaining: 2.76s
511:	learn: 0.4667852	total: 2.88s	remaining: 2.75s
512:	learn: 0.4666741	total: 2.89s	remaining: 2.74s
513:	learn: 0.4665425	total: 2.89s	remaining: 2.74s
514:	learn: 0.4663898	total: 2.9s	remaining: 2.73s
515:	learn: 0.4662856	total: 2.9s	remaining: 2.72s
516:	learn: 0.4661361	total: 2.91s	remaining: 2.72s
517:	learn: 0.4659074	total: 2.91s	remaining: 2.71s
518:	learn: 0.4657699	total: 2.92s	remaining: 2.7s
519:	learn: 0.4656599	total: 2.92s	remaining: 2.7s
520:	learn: 0.4654807	total: 2.92s	remaining: 2.69s
521:	learn: 0.4653629	total: 2.93s	remaining: 2.68s
522:	learn: 0.465

674:	learn: 0.4439704	total: 3.73s	remaining: 1.8s
675:	learn: 0.4438211	total: 3.74s	remaining: 1.79s
676:	learn: 0.4437318	total: 3.74s	remaining: 1.78s
677:	learn: 0.4435664	total: 3.75s	remaining: 1.78s
678:	learn: 0.4434831	total: 3.75s	remaining: 1.77s
679:	learn: 0.4433325	total: 3.76s	remaining: 1.77s
680:	learn: 0.4432108	total: 3.76s	remaining: 1.76s
681:	learn: 0.4430992	total: 3.77s	remaining: 1.76s
682:	learn: 0.4429837	total: 3.77s	remaining: 1.75s
683:	learn: 0.4428390	total: 3.78s	remaining: 1.75s
684:	learn: 0.4427333	total: 3.78s	remaining: 1.74s
685:	learn: 0.4425925	total: 3.79s	remaining: 1.73s
686:	learn: 0.4424629	total: 3.79s	remaining: 1.73s
687:	learn: 0.4422989	total: 3.79s	remaining: 1.72s
688:	learn: 0.4422310	total: 3.8s	remaining: 1.72s
689:	learn: 0.4421437	total: 3.8s	remaining: 1.71s
690:	learn: 0.4420060	total: 3.81s	remaining: 1.7s
691:	learn: 0.4418608	total: 3.81s	remaining: 1.7s
692:	learn: 0.4417423	total: 3.82s	remaining: 1.69s
693:	learn: 0.441

847:	learn: 0.4224018	total: 4.63s	remaining: 830ms
848:	learn: 0.4222591	total: 4.63s	remaining: 824ms
849:	learn: 0.4221253	total: 4.64s	remaining: 818ms
850:	learn: 0.4220562	total: 4.64s	remaining: 813ms
851:	learn: 0.4219743	total: 4.65s	remaining: 807ms
852:	learn: 0.4218327	total: 4.65s	remaining: 802ms
853:	learn: 0.4217496	total: 4.66s	remaining: 796ms
854:	learn: 0.4216224	total: 4.66s	remaining: 790ms
855:	learn: 0.4214883	total: 4.66s	remaining: 785ms
856:	learn: 0.4213712	total: 4.67s	remaining: 779ms
857:	learn: 0.4212902	total: 4.67s	remaining: 774ms
858:	learn: 0.4211746	total: 4.68s	remaining: 768ms
859:	learn: 0.4211052	total: 4.68s	remaining: 762ms
860:	learn: 0.4210047	total: 4.69s	remaining: 757ms
861:	learn: 0.4209389	total: 4.69s	remaining: 751ms
862:	learn: 0.4208320	total: 4.7s	remaining: 746ms
863:	learn: 0.4207367	total: 4.7s	remaining: 740ms
864:	learn: 0.4206009	total: 4.71s	remaining: 735ms
865:	learn: 0.4204876	total: 4.71s	remaining: 729ms
866:	learn: 0.

Precision: 0.36774193548387096
Accuracy: 0.6183685111237878
AUC: 0.500488322717622


In [35]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6897629	total: 3.4ms	remaining: 337ms
1:	learn: 0.6877999	total: 11.4ms	remaining: 561ms
2:	learn: 0.6857479	total: 13.4ms	remaining: 434ms
3:	learn: 0.6832526	total: 15.9ms	remaining: 382ms
4:	learn: 0.6817928	total: 19.4ms	remaining: 368ms
5:	learn: 0.6801554	total: 22.4ms	remaining: 350ms
6:	learn: 0.6758377	total: 25.8ms	remaining: 343ms
7:	learn: 0.6735926	total: 28.4ms	remaining: 327ms
8:	learn: 0.6696027	total: 30.6ms	remaining: 310ms
9:	learn: 0.6680994	total: 34.1ms	remaining: 307ms
10:	learn: 0.6665335	total: 35.7ms	remaining: 289ms
11:	learn: 0.6629852	total: 38.9ms	remaining: 285ms
12:	learn: 0.6615824	total: 41.3ms	remaining: 277ms
13:	learn: 0.6603282	total: 43.8ms	remaining: 269ms
14:	learn: 0.6586576	total: 46.1ms	remaining: 261ms
15:	learn: 0.6578663	total: 47.8ms	remaining: 251ms
16:	learn: 0.6550238	total: 49.4ms	remaining: 241ms
17:	learn: 0.6512519	total: 51.4ms	remaining: 234ms
18:	learn: 0.

--- Visualisation ---

In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
2          Tuned Decision Tree Precision:         0.434343
14                   Tuned XGB Precision:         0.405882
11            GradientBoosting Precision:         0.390244
12      Tuned GradientBoosting Precision:         0.390244
16                  Tuned LGBM Precision:         0.380000
1                Decision Tree Precision:         0.367868
17                    CatBoost Precision:         0.367742
13                         XGB Precision:         0.366771
7                Random Forest Precision:         0.356000
3         KNeighborsClassifier Precision:         0.348894
4   Tuned KNeighborsClassifier Precision:         0.346154
0          Logistic Regression Precision:         0.345977
15                        LGBM Precision:         0.343023
5                   GaussianNB Precision:         0.342074
9                     AdaBoost Precision:         0.338000
8          Tuned Random Forest Precision:         0.3363

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
11            GradientBoosting Accuracy:        0.636623
12      Tuned GradientBoosting Accuracy:        0.636623
16                  Tuned LGBM Accuracy:        0.634912
18              Tuned CatBoost Accuracy:        0.634341
2          Tuned Decision Tree Accuracy:        0.634341
14                   Tuned XGB Accuracy:        0.623503
10              Tuned AdaBoost Accuracy:        0.620080
17                    CatBoost Accuracy:        0.618369
15                        LGBM Accuracy:        0.610953
7                Random Forest Accuracy:        0.600685
8          Tuned Random Forest Accuracy:        0.600685
13                         XGB Accuracy:        0.593269
4   Tuned KNeighborsClassifier Accuracy:        0.582430
9                     AdaBoost Accuracy:        0.549344
6                          SVM Accuracy:        0.545351
1                Decision Tree Accuracy:        0.541358
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
2          Tuned Decision Tree AUC:   0.510743
1                Decision Tree AUC:   0.507953
7                Random Forest AUC:   0.506558
14                   Tuned XGB AUC:   0.505326
17                    CatBoost AUC:   0.500488
8          Tuned Random Forest AUC:   0.499001
4   Tuned KNeighborsClassifier AUC:   0.497670
13                         XGB AUC:   0.497346
15                        LGBM AUC:   0.490132
10              Tuned AdaBoost AUC:   0.489384
16                  Tuned LGBM AUC:   0.488836
6                          SVM AUC:   0.488819
3         KNeighborsClassifier AUC:   0.487079
9                     AdaBoost AUC:   0.486617
18              Tuned CatBoost AUC:   0.485099
12      Tuned GradientBoosting AUC:   0.484525
11            GradientBoosting AUC:   0.484525
0          Logistic Regression AUC:   0.482040
5                   GaussianNB AUC:   0.476862
